To generate processeed dataa from openpose
Videos : ./build/examples/openpose/openpose.bin --video <<path to videos file>> --write_video <<output_folder/>> 
Image : ./build/examples/openpose/openpose.bin --image_dir <<path_to_images folder> --write_images <path to output folder>> 
    
Output json with keypoints : 
./build/examples/openpose/openpose.bin --image_dir <<path_to_images folder> --write_json <path to output folder>>
    
For COCO
Videos : ./build/examples/openpose/openpose.bin --video <<path to videos file>> --model_pose COCO --write_video <<output_folder/>> 
Image : ./build/examples/openpose/openpose.bin --model_pose COCO --image_dir <<path_to_images folder> --write_images <path to output folder>> 
    
Output json with keypoints : 
./build/examples/openpose/openpose.bin --model_pose COCO --image_dir <<path_to_images folder> --write_json <path to output folder>>

In [3]:
import json
import pandas as pd

with open('/Users/jalpanchal/drive/penn/robo599/simulator_media/0331/out_json/0331_1_000000000001_keypoints.json', 'r') as file:
     kp_0331_1 = json.load(file)

In [4]:
kp_0331_1

{'version': 1.3,
 'people': [{'person_id': [-1],
   'pose_keypoints_2d': [335.504,
    290.145,
    0.884714,
    318.058,
    433.199,
    0.869849,
    188.923,
    433.408,
    0.721565,
    77.2669,
    321.516,
    0.729503,
    119.227,
    262.257,
    0.739111,
    440.368,
    433.247,
    0.805871,
    527.5,
    572.995,
    0.85514,
    499.696,
    653.162,
    0.879719,
    213.383,
    761.388,
    0.521734,
    122.696,
    963.909,
    0.845496,
    199.341,
    1054.5,
    0.753254,
    422.868,
    747.481,
    0.579554,
    579.89,
    907.906,
    0.820291,
    478.581,
    1016.16,
    0.808243,
    290.126,
    269.17,
    0.918264,
    366.941,
    262.397,
    0.928788,
    234.254,
    325.164,
    0.827336,
    408.906,
    307.623,
    0.692675],
   'face_keypoints_2d': [],
   'hand_left_keypoints_2d': [],
   'hand_right_keypoints_2d': [],
   'pose_keypoints_3d': [],
   'face_keypoints_3d': [],
   'hand_left_keypoints_3d': [],
   'hand_right_keypoints_3d': [

In [85]:
#from https://medium.com/@sduxbury/how-you-can-build-practical-applications-by-quantifying-observations-from-video-e266b945eea0

import numpy as np
import os
import cv2
import json
print('OpenCV - version: ',cv2.__version__)
import pandas as pd

# video file
cap = cv2.VideoCapture('/Users/jalpanchal/drive/penn/robo599/simulator_media/0331/raw_videos/0331_1.mp4')


def get_vid_properties(): 
    width = int(cap.get(3))  # float
    height = int(cap.get(4)) # float
    cap.release()
    return width,height
  
print('Video Dimensions: ',get_vid_properties())

# Load keypoint data from JSON output
column_names = ['x', 'y', 'acc', 'frame']

# Paths - should be the folder where Open Pose JSON output was stored
path_to_json = "/Users/jalpanchal/drive/penn/robo599/simulator_media/0331/out_json/"

# Import Json files, pos_json = position JSON
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print('Found: ',len(json_files),'json keypoint frame files')
count = 0

#sort files
json_files = np.sort(json_files)

width,height = get_vid_properties()

# instanciate dataframes 
body_keypoints_df = pd.DataFrame()
left_knee_df = pd.DataFrame()

print('json files: ',json_files[0])   

# Loop through all json files in output directory
# Each file is a frame in the video
# If multiple people are detected - choose the most centered high confidence points

for file in json_files:
    
    temp_df = json.load(open(path_to_json+file))
    temp = []
    for k,v in temp_df['part_candidates'][0].items():
        
        # Single point detected
        if len(v) < 4:
            temp.append(v)
            #print('Extracted highest confidence points: ',v)
            
        # Multiple points detected
        elif len(v) > 4: 
            near_middle = width
            np_v = np.array(v)
            
            # Reshape to x,y,confidence
            np_v_reshape = np_v.reshape(int(len(np_v)/3),3)
            np_v_temp = []
            # compare x values
            for pt in np_v_reshape:
                if(np.absolute(pt[0]-width/2)<near_middle):
                    near_middle = np.absolute(pt[0]-width/2)
                    np_v_temp = list(pt)
         
            temp.append(np_v_temp)
            #print('Extracted highest confidence points: ',v[index_highest_confidence-2:index_highest_confidence+1])
        else:
            # No detection - record zeros
            temp.append([0,0,0])
            
    temp_df = pd.DataFrame(temp)
    temp_df = temp_df.fillna(0)
    temp_df['frame'] = int(file[7:19])

    try:
        prev_temp_df = temp_df
        body_keypoints_df= body_keypoints_df.append(temp_df)
        left_knee_df = left_knee_df.append(temp_df.iloc[13].astype(int))

    except:
        print('bad point set at: ', file)
        
body_keypoints_df.columns = column_names
left_knee_df.columns = column_names




body_keypoints_df.reset_index()
left_knee_df = left_knee_df.reset_index(drop = True)

print('length of merged keypoint set: ',body_keypoints_df.size)

print(left_knee_df.head())

OpenCV - version:  4.5.1
Video Dimensions:  (720, 1280)
Found:  50 json keypoint frame files
json files:  0331_1_000000000000_keypoints.json
length of merged keypoint set:  3600
       x       y  acc  frame
0  478.0  1016.0  0.0    0.0
1  478.0  1016.0  0.0    1.0
2  478.0  1016.0  0.0    2.0
3  478.0  1012.0  0.0    3.0
4  482.0  1012.0  0.0    4.0


In [89]:
body_keypoints_df.head(50)

,x,y,acc,frame
0,335.517,290.236,0.875829,0
1,314.647,429.896,0.860785,0
2,185.415,433.319,0.742240,0
3,0.000,0.000,0.000000,0
4,119.163,262.296,0.731809,0
5,440.279,433.262,0.798600,0
6,527.502,576.391,0.838653,0
7,499.667,649.784,0.856830,0
8,213.380,757.916,0.523818,0
9,122.639,963.834,0.863249,0
